**Sections:**<a name="contents"></a>

[Control data download](#control_data_download)

[Create symbolink link copies for all experimental fastq files](#symb_link_input)

[Quality control](#QC_analysis)

In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

import warnings
warnings.simplefilter('ignore')

# general purpose packages
import pandas as pd
import numpy as np
import os
import json
import time
import re
import csv
import subprocess
import sys

import scipy.stats as stats
import statsmodels.stats as smstats
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import umap
import rpy2

from multiprocessing import Process, Manager, Pool
import multiprocessing
from functools import partial

from collections import Counter

import seaborn as sns; sns.set()

import matplotlib
matplotlib.style.use('seaborn')
matplotlib.use('Agg')
import matplotlib.pyplot as plt
matplotlib.rcParams['backend'] = "Qt5Agg"
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter

from IPython.display import display, Image

from adjustText import adjust_text
import builtins
%matplotlib inline

# for normalization
from sklearn.linear_model import QuantileRegressor

# for working with yaml files
import ruamel.yaml

# for working with genomic intervals
import pyranges as pr

import itertools

In [2]:
def get_pvalue_star(pval, thr=0.05):
    if thr == 0.05:
        if pval < 0.001:
            return "***"
        elif pval < 0.01:
            return "**"
        elif pval < 0.05:
            return "*"
        else:
            return "ns"
    elif thr == 0.1:
        if pval < 0.001:
            return "***"
        elif pval < 0.01:
            return "**"
        elif pval < 0.1:
            return "*"
        else:
            return "ns"

In [22]:
# paths to subdirectories
subdirs = {}

subdirs['main_project_dir'] = '/scicore/home/zavolan/GROUP/RBP_perturbational_networks/'
subdirs['wf_dir'] = '/scicore/home/zavolan/mirono0000/Projects/RBP_perturbational_networks/WF/'

# tandem pas WF dir
subdirs['tandem_PAS_dir'] = '/scicore/home/zavolan/mirono0000/libs/tandem-pas/'

subdirs['human_annotation_dir'] = '/scicore/home/zavolan/GROUP/Genomes/homo_sapiens/'

# shared project folder 
subdirs['shared_project_dir'] = subdirs['main_project_dir']
subdirs['temp_dir'] = subdirs['shared_project_dir']+'temp_dir/'
subdirs['slurm_dir'] = subdirs['temp_dir']+'slurm/'
subdirs['scripts_dir'] = subdirs['shared_project_dir']+'scripts/'
subdirs['figures_dir'] = subdirs['shared_project_dir']+'figures/'
subdirs['fastq_dir'] = subdirs['shared_project_dir']+'input_fastq/'
subdirs['metadata_dir'] = subdirs['shared_project_dir']+'metadata/'

subdirs['wf_runs_dir'] = subdirs['shared_project_dir']+'wf_runs/'

# paths to files
file_paths = {}
### genome annotation files
file_paths['human_genome_file'] = subdirs['human_annotation_dir']+'GRCh38.primary_assembly.genome.fa'
file_paths['human_annotation_file'] = subdirs['human_annotation_dir']+'hg38_v42/gencode.v42.annotation.gtf'
file_paths['human_RNAcentral_annotation_file'] = subdirs['human_annotation_dir']+'hg38_v42/homo_sapiens.GRCh38.gff3.gz'
file_paths['human_enriched_annotation_file'] = subdirs['human_annotation_dir']+'hg38_v42/enriched.gencode.v42.annotation.gtf'

file_paths['human_polyAsite_atlas'] = subdirs['human_annotation_dir']+'hg38_v42/atlas.clusters.2.0.GRCh38.96.bed.gz'

os.system('mkdir -p '+' '.join(list(subdirs.values()))) # create all subdirs

0

# Make enriched gtf file for mouse and human

In [29]:
organisms = ['human']

for organism in organisms:
    command = 'samtools faidx '+file_paths[organism+'_genome_file']
    out = subprocess.check_output(command, shell=True)

    gtf_df = pd.read_csv(file_paths[organism+'_annotation_file'],delimiter="\t",index_col=None,header=None,skiprows=5)

    rna_central = pd.read_csv(file_paths[organism+'_RNAcentral_annotation_file'],delimiter="\t",index_col=None,header=None,skiprows=1,compression='gzip')
    rna_central[2] = rna_central[2].str.replace('noncoding_exon','exon')
    rna_central['gene_biotype'] = rna_central[8].str.split(';type=|;',expand=True)[1]
    rna_central['gene_source'] = 'RNA_central'
    rna_central['gene_id'] = rna_central[8].str.split('ID=|;|:',expand=True)[4]

    rna_central_exons = rna_central.loc[rna_central[2]=='exon'].copy().reset_index(drop=True)
    rna_central_exons['exon_id'] = rna_central_exons['gene_id']+'.transcript'+'.'+rna_central_exons[8].str.split('ID=|;|:',expand=True)[5]
    rna_central_exons['exon_number'] = rna_central_exons[8].str.split('ID=|;|:',expand=True)[5].str.split('exon',expand=True).iloc[:, -1]
    rna_central_exons[8] = 'gene_id "'+rna_central_exons['gene_id']+'"; transcript_id "'+rna_central_exons['gene_id']+'.transcript'+'"; exon_number "'+rna_central_exons['exon_number']+'"; gene_source "'+rna_central_exons['gene_source']+'"; gene_type "'+rna_central_exons['gene_biotype']+'"; transcript_source "'+rna_central_exons['gene_source']+'"; transcript_type "'+rna_central_exons['gene_biotype']+'"; exon_id "'+rna_central_exons['exon_id']+'"; tag "'+rna_central['gene_source']+'";'
    rna_central_exons['order']=3

    rna_central_exons['transcript_id'] = rna_central_exons['gene_id']+'.transcript'
    rna_central_exons['exon_coords'] = rna_central_exons[3].astype('str')+'_'+rna_central_exons[4].astype('str')+','
    rna_central_gr_transcripts = rna_central_exons.groupby([0,6,'transcript_id']).agg({'exon_coords':sum}).reset_index()
    rna_central_gr_transcripts['transcript_alt_id'] = rna_central_gr_transcripts[0].astype('str')+'_'+rna_central_gr_transcripts[6]+'_'+rna_central_gr_transcripts['exon_coords']

    gtf_df_exons = gtf_df.loc[gtf_df[2]=='exon'].reset_index(drop=True)
    gtf_df_exons['transcript_id'] = gtf_df_exons[8].str.split('transcript_id "',expand=True)[1].str.split('"',expand=True)[0]
    gtf_df_exons['exon_coords'] = gtf_df_exons[3].astype('str')+'_'+gtf_df_exons[4].astype('str')+','
    gtf_df_gr_transcripts = gtf_df_exons.groupby([0,6,'transcript_id']).agg({'exon_coords':sum}).reset_index()
    gtf_df_gr_transcripts['transcript_alt_id'] = gtf_df_gr_transcripts[0].astype('str')+'_'+gtf_df_gr_transcripts[6]+'_'+gtf_df_gr_transcripts['exon_coords']
    ensembl_transcripts = list(gtf_df_gr_transcripts['transcript_alt_id'].unique())

    preserve_transcripts_list = list(rna_central_gr_transcripts.loc[~rna_central_gr_transcripts['transcript_alt_id'].isin(ensembl_transcripts)]['transcript_id'].unique()) # when a transcript is present in ensemble and RNA central, prioritize ensemble
    rna_central_exons = rna_central_exons.loc[rna_central_exons['transcript_id'].isin(preserve_transcripts_list)].reset_index(drop=True)
    rna_central['transcript_id'] = rna_central['gene_id']+'.transcript'
    rna_central = rna_central.loc[rna_central['transcript_id'].isin(preserve_transcripts_list)].reset_index(drop=True)

    rna_central_exons = rna_central_exons[list(range(0,9))+['gene_id','order']]

    rna_central_transcripts = rna_central.loc[rna_central[2]=='transcript'].copy().reset_index(drop=True)
    rna_central_transcripts[8] = 'gene_id "'+rna_central_transcripts['gene_id']+'"; transcript_id "'+rna_central_transcripts['gene_id']+'.transcript'+'"; gene_source "'+rna_central_transcripts['gene_source']+'"; gene_type "'+rna_central_transcripts['gene_biotype']+'"; transcript_source "'+rna_central_transcripts['gene_source']+'"; transcript_type "'+rna_central_transcripts['gene_biotype']+'"; tag "'+rna_central_transcripts['gene_source']+'";'
    rna_central_transcripts['order']=2
    rna_central_transcripts = rna_central_transcripts[list(range(0,9))+['gene_id','order']]

    rna_central_genes = rna_central.loc[rna_central[2]=='transcript'].copy().reset_index(drop=True)
    rna_central_genes[2] = 'gene'
    rna_central_genes[8] = 'gene_id "'+rna_central_genes['gene_id']+'"; gene_source "'+rna_central_genes['gene_source']+'"; gene_type "'+rna_central_genes['gene_biotype']+'";'
    rna_central_genes['order']=1
    rna_central_genes = rna_central_genes[list(range(0,9))+['gene_id','order']]

    rna_central_gtf = pd.concat([rna_central_genes,rna_central_transcripts,rna_central_exons]).sort_values(['gene_id','order']).reset_index(drop=True).drop(['gene_id','order'],1)
    rna_central_gtf[0] = 'chr'+rna_central_gtf[0].astype('str') # so that the naming is consistent
    # save standard annotation enriched with RNA central
    enriched_gtf = pd.concat([gtf_df,rna_central_gtf]).reset_index(drop=True)
    genome_fai = pd.read_csv(file_paths[organism+'_genome_file']+'.fai',delimiter="\t",index_col=None,header=None)
    enriched_gtf = pd.merge(genome_fai[[0]],enriched_gtf,how='inner',on=0)

    # remove non-canonical chromosomes, they are not of interest
    enriched_gtf = enriched_gtf.loc[enriched_gtf[0].str.startswith('chr')].reset_index(drop=True)
    enriched_gtf.to_csv(file_paths[organism+'_enriched_annotation_file'], sep=str('\t'),header=False,index=None,quoting=csv.QUOTE_NONE)

# Enrich ENCODE metadata using json files from ENCODE portal

In [4]:
ENCODE_metadata = pd.read_csv(subdirs['metadata_dir']+'ENCODE_metadata.tsv',delimiter="\t",
                                   index_col=None,header=0)

# add information about where are right controls
ENCODE_metadata['json_url'] = """https://www.encodeproject.org"""+ENCODE_metadata['File dataset']+"""?format=json"""
ENCODE_metadata[['json_url']].drop_duplicates().to_csv(subdirs['metadata_dir']+'ENCODE_json_urls.txt', sep=str('\t'),header=False,index=None,quoting=csv.QUOTE_NONE)

command = 'mkdir -p '+subdirs['metadata_dir']+'ENCODE_json_files/'
out = subprocess.check_output(command, shell=True)

command = """wget -i """+subdirs['metadata_dir']+'ENCODE_json_urls.txt -P '+subdirs['metadata_dir']+'ENCODE_json_files/'
print(command)

os.system("""find """+subdirs['metadata_dir']+'ENCODE_json_files/'+""" -name '*=json*' > """+subdirs['temp_dir']+"""json_file_paths.tsv""")
json_file_paths = pd.read_csv(subdirs['temp_dir']+'json_file_paths.tsv',delimiter="\t",
                                   index_col=None,header=None)
a = []
for json_file_path in list(json_file_paths[0]):
    json = pd.read_json(json_file_path,orient='index')
    if len(json.loc['possible_controls'][0])>0:
        a.append([json.loc['accession'][0],json.loc['possible_controls'][0][0]['accession']])
    else:
        a.append([json.loc['accession'][0],''])
controls = pd.DataFrame(a,columns= ['File dataset','File dataset controls'])
controls['File dataset'] = '/experiments/'+controls['File dataset']+'/'

ENCODE_metadata = pd.merge(ENCODE_metadata,controls,how='left',on=['File dataset'])

# add information about which file corresponds to read 1, and which - to read 2

ENCODE_metadata['file_json_url'] = """https://www.encodeproject.org/files/"""+ENCODE_metadata['File accession']+"""/?format=json"""
ENCODE_metadata[['file_json_url']].drop_duplicates().to_csv(subdirs['metadata_dir']+'ENCODE_fastq_json_urls.txt', sep=str('\t'),header=False,index=None,quoting=csv.QUOTE_NONE)

command = 'mkdir -p '+subdirs['metadata_dir']+'ENCODE_fastq_json_files/'
out = subprocess.check_output(command, shell=True)

command = """wget -i """+subdirs['metadata_dir']+'ENCODE_fastq_json_urls.txt -P '+subdirs['metadata_dir']+'ENCODE_fastq_json_files/'
print(command)

os.system("""find """+subdirs['metadata_dir']+'ENCODE_fastq_json_files/'+""" -name '*=json*' > """+subdirs['temp_dir']+"""json_fastq_file_paths.tsv""")
json_fastq_file_paths = pd.read_csv(subdirs['temp_dir']+'json_fastq_file_paths.tsv',delimiter="\t",
                                   index_col=None,header=None)

a = []
for json_file_path in list(json_fastq_file_paths[0]):
    json = pd.read_json(json_file_path,orient='index')
    a.append([json.loc['accession'][0],json.loc['paired_with'][0],json.loc['paired_end'][0],json.loc['biological_replicates_formatted'][0],json.loc['read_length'][0]])
file_metadata = pd.DataFrame(a,columns= ['File accession','paired_with','paired_end','bioreplicate','read_length'])
file_metadata['paired_with'] = file_metadata['paired_with'].str.replace('/files/','').str.replace('/','')

ENCODE_metadata = pd.merge(ENCODE_metadata,file_metadata,how='left',on=['File accession'])

def get_sample(x):
    l = [x['File accession'],x['paired_with']]
    l.sort()
    return '_'.join(l)
ENCODE_metadata['sample'] = ENCODE_metadata.apply(lambda x: get_sample(x),1)

ENCODE_metadata['File dataset'] = ENCODE_metadata['File dataset'].str.replace('/experiments/','').str.replace('/','')

EXPS = ENCODE_metadata.loc[~ENCODE_metadata['File target'].isna()].reset_index(drop=True)
EXPS['experiment_id'] = EXPS['File dataset']+'_'+EXPS['File dataset controls']+';'+EXPS['Biosample term name']+';'+EXPS['File target']

CONTROLS = ENCODE_metadata.loc[ENCODE_metadata['File target'].isna()].reset_index(drop=True)
CONTROLS['File dataset controls'] = CONTROLS['File dataset']
tmp = pd.merge(EXPS[['sample','experiment_id','Biosample term name','File target','File dataset controls']],CONTROLS[['File dataset controls','sample']],how='inner',on='File dataset controls')

df = pd.melt(tmp,id_vars=['experiment_id','Biosample term name','File target'],value_vars=['sample_x','sample_y'],value_name='sample')[['sample','experiment_id','Biosample term name','File target']].drop_duplicates()

# prepare the start samples table in which rows correspond to samples (like in SRA metadata) and experiment ids are provided
encode_start_samples = pd.merge(df,ENCODE_metadata.loc[ENCODE_metadata['paired_end']=='1'][['sample','read_length','File target','bioreplicate','Assay term name','File download URL']].rename(columns={'File download URL':'fq1','File target':'EXP_CTL'}),how='left',on='sample')
encode_start_samples = pd.merge(encode_start_samples,ENCODE_metadata.loc[ENCODE_metadata['paired_end']=='2'][['sample','File download URL']].rename(columns={'File download URL':'fq2'}),how='left',on='sample')

encode_start_samples = encode_start_samples[['sample','fq1','fq2','read_length','bioreplicate','EXP_CTL','experiment_id','Biosample term name','File target','Assay term name']]
encode_start_samples['experiment_id'] = encode_start_samples['experiment_id']+';'+(encode_start_samples['Assay term name'].str.contains('shRNA')).astype('str').str.replace('True','KD').replace('False','KO')
encode_start_samples['EXP_CTL'] = (encode_start_samples['EXP_CTL'].isna()).astype('str').str.replace('True','CTL').replace('False','EXP')
encode_start_samples = encode_start_samples.sort_values(['experiment_id','EXP_CTL']).reset_index(drop=True)
encode_start_samples = encode_start_samples.rename(columns = {'File target':'targeted_gene','Biosample term name':'cell_line'})
encode_start_samples['assay'] = (encode_start_samples['Assay term name'].str.contains('shRNA')).astype('str').str.replace('True','shRNA').replace('False','CRISPR')

encode_start_samples['genome_file'] = file_paths['human_genome_file']
encode_start_samples['gtf_file'] = file_paths['human_enriched_annotation_file']
encode_start_samples['organism'] = 'human'
encode_start_samples = encode_start_samples[['sample','fq1','fq2','read_length','organism','genome_file','gtf_file','experiment_id','targeted_gene','bioreplicate','EXP_CTL','cell_line','assay']]

encode_start_samples.to_csv(subdirs['metadata_dir']+'encode_start_samples.tsv', sep=str('\t'),header=True,index=None,quoting=csv.QUOTE_NONE)

wget -i /scicore/home/zavolan/GROUP/RBP_perturbational_networks/metadata/ENCODE_json_urls.txt -P /scicore/home/zavolan/GROUP/RBP_perturbational_networks/metadata/ENCODE_json_files/
wget -i /scicore/home/zavolan/GROUP/RBP_perturbational_networks/metadata/ENCODE_fastq_json_urls.txt -P /scicore/home/zavolan/GROUP/RBP_perturbational_networks/metadata/ENCODE_fastq_json_files/


In [21]:
len(encode_start_samples.drop_duplicates('sample'))

1060

# Get Tandem Poly(A) Sites for organisms

In [21]:
# add 'chr' to the polyAsite Atlas file

command = """zcat """+file_paths['human_polyAsite_atlas']+r""" | sed -E 's/^([0-9]+|[XYM])/chr\1/' | pigz --stdout > """+file_paths['human_polyAsite_atlas'].replace('.bed.gz','.wchr.bed.gz')+\
""" && mv """+file_paths['human_polyAsite_atlas'].replace('.bed.gz','.wchr.bed.gz')+' '+file_paths['human_polyAsite_atlas']
print(command)
# run the command in terminal

zcat /scicore/home/zavolan/GROUP/Genomes/homo_sapiens/hg38_v42/atlas.clusters.2.0.GRCh38.96.bed.gz | sed -E 's/^([0-9]+|[XYM])/chr\1/' | pigz --stdout > /scicore/home/zavolan/GROUP/Genomes/homo_sapiens/hg38_v42/atlas.clusters.2.0.GRCh38.96.wchr.bed.gz && mv /scicore/home/zavolan/GROUP/Genomes/homo_sapiens/hg38_v42/atlas.clusters.2.0.GRCh38.96.wchr.bed.gz /scicore/home/zavolan/GROUP/Genomes/homo_sapiens/hg38_v42/atlas.clusters.2.0.GRCh38.96.bed.gz


In [57]:
# modify .yaml file for the tandem-pas pipeline

yaml = ruamel.yaml.YAML()
yaml.preserve_quotes = True
with open(subdirs['tandem_PAS_dir']+'configs/config.yaml') as f_read:
    data = yaml.load(f_read)

data['polyasites'] = file_paths['human_polyAsite_atlas']
data['atlas_version'] = 'v2.0'
data['gtf'] = file_paths['human_enriched_annotation_file']

with open(subdirs['tandem_PAS_dir']+'configs/config_hg38.yaml','w') as f_write:     
    yaml.dump(data, f_write)

In [58]:
subdirs['tandem_PAS_dir']

'/scicore/home/zavolan/mirono0000/libs/tandem-pas/'

In [65]:
# run tandem-pas WF with activated env

command = """snakemake \
--snakefile Snakefile \
--configfile configs/config_hg38.yaml \
--printshellcmds \
--use-singularity \
--singularity-args "--bind ${PWD}/../,"""+subdirs['human_annotation_dir']+"""" \
--cluster-config configs/cluster_config.json \
--cores 500 \
--local-cores 10 \
--cluster "sbatch \
--cpus-per-task {cluster.threads} \
--mem {cluster.mem} \
--qos {cluster.queue} \
--time {cluster.time} \
-o {params.cluster_log} \
--export=JOB_NAME={rule} \
--open-mode=append" \
--jobs 20 \
--nolock \
-np"""
command

'snakemake --snakefile Snakefile --configfile configs/config_hg38.yaml --printshellcmds --use-singularity --singularity-args "--bind ${PWD}/../,/scicore/home/zavolan/GROUP/Genomes/homo_sapiens/" --cluster-config configs/cluster_config.json --cores 500 --local-cores 10 --cluster "sbatch --cpus-per-task {cluster.threads} --mem {cluster.mem} --qos {cluster.queue} --time {cluster.time} -o {params.cluster_log} --export=JOB_NAME={rule} --open-mode=append" --jobs 20 --nolock -np'

# Prepare .yaml config file

In [155]:
# load default rule_config, modify it and save

WF_version = 'v1'

yaml = ruamel.yaml.YAML()
yaml.preserve_quotes = True
with open(subdirs['wf_dir']+'config.yaml') as f_read:
    data = yaml.load(f_read)
data['samples_file'] = subdirs['metadata_dir']+'encode_start_samples.tsv'

data['output_dir'] = subdirs['wf_runs_dir']+WF_version+'/output/'
data['local_log'] = subdirs['wf_runs_dir']+WF_version+'/output/local_log/'
data['cluster_log'] = subdirs['wf_runs_dir']+WF_version+'/output/cluster_log/'

command = 'mkdir -p '+subdirs['wf_runs_dir']+WF_version+'/output/'
out = subprocess.check_output(command, shell=True)

with open(subdirs['wf_runs_dir']+WF_version+'/config.yaml','w') as f_write:     
    yaml.dump(data, f_write)

In [161]:
subdirs['wf_runs_dir']+WF_version+'/output/cluster_log/'

'/scicore/home/zavolan/GROUP/RBP_perturbational_networks/wf_runs/v1/output/cluster_log/'

# Run main WF

In [32]:
subdirs['wf_dir']

'/scicore/home/zavolan/mirono0000/Projects/RBP_perturbational_networks/WF/'

In [167]:
WF_version = 'v1'

command = """snakemake \
--snakefile Snakefile \
--configfile """+subdirs['wf_runs_dir']+WF_version+'/config.yaml'+""" \
--printshellcmds \
--use-conda --conda-frontend conda \
--use-singularity \
--singularity-args "--bind """+subdirs['main_project_dir']+','+subdirs['human_annotation_dir']+"""" \
--cluster-config cluster.json \
--cores 500 \
--local-cores 10 \
--jobs 100 \
--latency-wait 60 \
--cluster "sbatch \
--cpus-per-task={cluster.threads} \
--mem={cluster.mem} \
--qos={cluster.queue} \
--partition={cluster.partition} \
--time={cluster.time} \
--output={cluster.out}" \
--nolock \
-np"""
command

'snakemake --snakefile Snakefile --configfile /scicore/home/zavolan/GROUP/RBP_perturbational_networks/wf_runs/v1/config.yaml --printshellcmds --use-conda --conda-frontend conda --use-singularity --singularity-args "--bind /scicore/home/zavolan/GROUP/RBP_perturbational_networks/,/scicore/home/zavolan/GROUP/Genomes/homo_sapiens/" --cluster-config cluster.json --cores 500 --local-cores 10 --jobs 100 --latency-wait 60 --cluster "sbatch --cpus-per-task={cluster.threads} --mem={cluster.mem} --qos={cluster.queue} --partition={cluster.partition} --time={cluster.time} --output={cluster.out}" --nolock -np'